In [247]:
import sqlite3
import pandas as pd
import numpy as np
import json

# Data exporting

In [248]:
# ### Convert .sqlite to .csv ###

# def sqlite2csv(input_filename, output_filename):
#     db = sqlite3.connect(input_filename)
#     cursor = db.cursor()
#     cursor.execute('SELECT name FROM sqlite_master WHERE type="table";')
#     tables = cursor.fetchall()
#     for table_name in tables:
#         table_name = table_name[0]
#         table = pd.read_sql_query('SELECT * FROM %s' % table_name, db)
#         table.to_csv(table_name + '.csv', index_label='index')
#     cursor.close()
#     db.close()
#     del table
#     table_names = pd.DataFrame(tables)
#     table_names.to_csv(output_filename)
#     return table_names

folder = 'data/'
database_filename = folder + 'database.sqlite'
csv_filename = folder + 'table_names.csv'
# table_names = sqlite2csv(database_filename, csv_filename)

In [249]:
### Put .csv files into dataframes ###

table_names = pd.read_csv(csv_filename, index_col=0)
names = [name[0] for name in table_names.values]

# Create dictionary
df = {}
for name in names:
    df[name] = pd.read_csv(folder + name + '.csv', index_col=0)
    del df[name].index.name

/home/zhangvwk/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/zhangvwk/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [250]:
def dump_dict(dict_to_be_dumped, dict_name):
    with open(dict_name + '.json', 'w') as f:
        json.dump(dict_to_be_dumped, f)

# Data cleaning 
## Dataframe: 'user'

In [251]:
df['user'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62593 entries, 0 to 62592
Data columns (total 22 columns):
id              62593 non-null int64
first_name      62593 non-null object
last_name       62593 non-null object
city            61163 non-null object
country         62495 non-null object
sex             62593 non-null int64
height          62593 non-null int64
weight          62593 non-null int64
started         62593 non-null int64
competitions    4888 non-null object
occupation      11778 non-null object
sponsor1        5355 non-null object
sponsor2        2656 non-null object
sponsor3        1497 non-null object
best_area       15413 non-null object
worst_area      220 non-null object
guide_area      7755 non-null object
interests       8577 non-null object
birth           27856 non-null object
presentation    6113 non-null object
deactivated     62593 non-null int64
anonymous       62593 non-null int64
dtypes: int64(7), object(15)
memory usage: 11.0+ MB


In [252]:
### Drop useless user features ###

useless_user_columns = ['first_name',
                        'last_name',
                        'city', 
                        'competitions', 
                        'occupation', 
                        'sponsor1', 
                        'sponsor2', 
                        'sponsor3', 
                        'best_area', 
                        'worst_area', 
                        'guide_area', 
                        'interests', 
                        'presentation', 
                        'deactivated', 
                        'anonymous']

df['user'].drop(columns=useless_user_columns, inplace=True)

In [253]:
### COUNTRY ###

# Drop users with missing country
df['user'].dropna(subset=['country'], inplace=True)

# Drop users with invalid country
df['user'] = df['user'][df['user']['country'].map(type) == str]
df['user'] = df['user'][df['user']['country'].map(len) == 3]

# Convert country to index
countries = df['user']['country'].unique()
country2idx = {}
idx = 0
for country in df['user']['country'].unique():
    country2idx[country] = idx
    df['user']['country'][df['user']['country'] == country] = idx
    idx += 1
    
# Dump country2idx
dump_dict(country2idx, 'country2idx')

/home/zhangvwk/.local/lib/python3.5/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [254]:
### SEX ###

# 0 is male, 1 is female
df['user'].rename(columns={'sex':'is_female'}, inplace=True)
# 255 is unlikely to be non-binary so drop 255
df['user'].drop(index=list(df['user'][df['user']['is_female']==255].index), inplace=True)

In [255]:
### BIRTH AND STARTED ###

df['user']['birth'] = pd.to_datetime(df['user']['birth'], errors='coerce')
df['user']['birth'] = [date.year for date in df['user']['birth']]
df['user']['started'] = pd.to_datetime(df['user']['started'], errors='coerce', format='%Y')
df['user']['started'] = [date.year for date in df['user']['started']]

df['user'] = df['user'][df['user']['birth'] > 1940]

In [256]:
### HEIGHT AND WEIGHT ###

# Replace 0 values with null
df['user']['height'].replace(0, np.nan, inplace=True)
df['user']['weight'].replace(0, np.nan, inplace=True)
# df['user']['weight'] = df['user']['weight'].dropna()

# Drop rows with null values in both height and weight
# Since it's too hard to infer the correct values for either one based on the other features only
# NOTE: losing around 50% of the dataset as a consequence
df['user'].dropna(subset=['height', 'weight'], how='all', inplace=True)

# Filter height to values between 140cm and 240cm
df['user'] = df['user'][df['user']['height'] >= 140]
df['user'] = df['user'][df['user']['height'] <= 240]

# at this point, ~29k rows

In [257]:
# Extract a complete dataset
df_complete = df['user'].dropna(how='any')
from sklearn.model_selection import train_test_split
# Split the complete dataset into train/test sets (80/20)
df_train_orig, df_test = train_test_split(df_complete, test_size=0.2, random_state=229)

# Apply additional bogus checks on both train and test sets
df_train_orig['age_when_started_climbing'] = df_train_orig['started'] - df_train_orig['birth']
df_train_orig = df_train_orig[df_train_orig['age_when_started_climbing'] >= 10]

df_test['age_when_started_climbing'] = df_test['started'] - df_test['birth']
df_test = df_test[df_test['age_when_started_climbing'] >= 10]

# Drop weight as it is correlated with height
df_train_orig.drop(columns='weight', inplace=True)
df_test.drop(columns='weight', inplace=True)

/home/zhangvwk/.local/lib/python3.5/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/zhangvwk/.local/lib/python3.5/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [258]:
print('There are {} unique users in train_orig.'.format(len(df_train_orig['id'].unique())))
print('There are {} unique users in test.'.format(len(df_test['id'].unique())))
df_train_orig.head()

There are 14063 unique users in train_orig.
There are 3521 unique users in test.


,id,country,is_female,height,started,birth,age_when_started_climbing
4115,4275,21,0,176.0,2000.0,1974.0,26.0
31006,32351,12,0,175.0,2000.0,1989.0,11.0
21852,22763,14,0,175.0,2007.0,1988.0,19.0
27360,28503,21,0,159.0,1996.0,1976.0,20.0
42850,45465,21,0,180.0,2012.0,1984.0,28.0


In [259]:
### Interpolation on missing height data

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

incomplete = df['user'].drop(columns=['id', 'started'])
imp = IterativeImputer(max_iter=10, random_state=0)
# Infer the missing data on: country, is_female, height, weight, birth
imp.fit(incomplete)
complete = pd.DataFrame(imp.transform(incomplete), columns=['country', 'is_female', 'height', 'weight', 'birth'])

# Match the indexing
complete.index = df['user'].index

In [260]:
# # Replace the original with the interpolated height column
complete = complete.astype(int)

df['user']['height'] = complete['height']
df['user']['birth'] = complete['birth']
# # Drop the weight column since it is correlated with the height
df['user'].drop(columns=['weight'], inplace=True)
# print(df['user'])

In [261]:
incomplete_started = df['user'].drop(columns=['id'])
imp_started = IterativeImputer(max_iter=10, random_state=0)
# Infer the missing started data on: country, is_female, height, weight, started, birth
imp_started.fit(incomplete_started)
complete_started = pd.DataFrame(imp_started.transform(incomplete_started), columns=['country', 'is_female', 'height', 'started', 'birth'])

# Match the indexing
complete_started.index = df['user'].index

In [262]:
# # Replace the original with the interpolated height column
complete_started = complete_started.astype(int)
# complete_started
df['user']['started'] = complete_started['started']
# print(df['user'])

# Add age_when_started_climbing column and drop those with values < 10 (years old)
df['user']['age_when_started_climbing'] = df['user']['started'] - df['user']['birth']
df['user'] = df['user'][df['user']['age_when_started_climbing'] >= 10]

In [263]:
# Checkpoint
checkpoint_df_user = df['user']
df_train_imp = checkpoint_df_user.merge(df_test, how='outer', indicator=True)
df_train_imp = df_train_imp[df_train_imp['_merge'] == 'left_only']
df_train_imp = df_train_imp.drop(columns='_merge')

In [264]:
print('There are {} unique users in train_imp.'.format(len(df_train_imp['id'].unique())))

There are 18403 unique users in train_imp.


## Dataframe: 'ascent'

In [265]:
df['ascent'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 4111878 entries, 0 to 4111876
Data columns (total 28 columns):
id                      int64
user_id                 float64
grade_id                int64
notes                   object
raw_notes               int64
method_id               int64
climb_type              int64
total_score             int64
date                    float64
year                    float64
last_year               float64
rec_date                float64
project_ascent_date     float64
name                    object
crag_id                 float64
crag                    object
sector_id               float64
sector                  object
country                 object
comment                 object
rating                  float64
description             object
yellow_id               float64
climb_try               float64
repeat                  float64
exclude_from_ranking    float64
user_recommended        float64
chipped                 float64
dtypes: float64

In [266]:
useless_ascent_columns = ['notes', 
                          'raw_notes',
                          'total_score', 
                          'date', 
                          'last_year', 
                          'rec_date', 
                          'project_ascent_date', 
                          'name', 
                          'crag_id', 
                          'crag', 
                          'sector_id', 
                          'sector', 
                          'country', 
                          'comment', 
                          'rating', 
                          'description', 
                          'yellow_id', 
                          'climb_try', 
                          'repeat', 
                          'exclude_from_ranking', 
                          'user_recommended', 
                          'chipped']

df['ascent'].drop(columns=useless_ascent_columns, inplace=True)

In [267]:
df['ascent'] = df['ascent'].dropna().astype(int)
df['ascent'].rename(columns={'climb_type':'is_bouldering'}, inplace=True)

In [268]:
df['ascent']['method_id'].unique()

array([3, 1, 2, 5, 4])

In [269]:
# Set method_id=5 to 3 since both designate onsight
df['ascent']['method_id'][df['ascent']['method_id'] == 5] = 3
# Set all other methods to be 0, as in not onsight
df['ascent']['method_id'][df['ascent']['method_id'] != 3] = 0
# Set method_id=3 to 1 as in onsight
df['ascent']['method_id'][df['ascent']['method_id'] == 3] = 1
# Rename 'method_id' as 'is_onsight' with now binary values only
df['ascent'].rename(columns={'method_id':'is_onsight'}, inplace=True)

In [270]:
# Checkpoint
checkpoint_df_ascent = df['ascent']
checkpoint_df_ascent.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4111877 entries, 0 to 4111876
Data columns (total 6 columns):
id               int64
user_id          int64
grade_id         int64
is_onsight       int64
is_bouldering    int64
year             int64
dtypes: int64(6)
memory usage: 219.6+ MB


# Dataframe: 'grade'

In [271]:
df['grade'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83 entries, 0 to 82
Data columns (total 14 columns):
id                       83 non-null int64
score                    83 non-null int64
fra_routes               83 non-null object
fra_routes_input         83 non-null int64
fra_routes_selector      83 non-null int64
fra_boulders             83 non-null object
fra_boulders_input       83 non-null int64
fra_boulders_selector    83 non-null int64
usa_routes               47 non-null object
usa_routes_input         83 non-null int64
usa_routes_selector      83 non-null int64
usa_boulders             45 non-null object
usa_boulders_input       83 non-null int64
usa_boulders_selector    83 non-null int64
dtypes: int64(10), object(4)
memory usage: 9.7+ KB


In [272]:
# Create a id to grade dict for later use
id2grade = pd.Series(df['grade'][['id','fra_routes']].fra_routes.values,index=df['grade'][['id','fra_routes']].id).to_dict()
dump_dict(id2grade, 'id2grade')

In [273]:
useless_grade_columns = ['score',
                        'fra_routes',
                        'fra_routes_input',
                        'fra_routes_selector',
                        'fra_boulders',
                        'fra_boulders_input',
                        'fra_boulders_selector',
                        'usa_routes',
                        'usa_routes_input',
                        'usa_routes_selector',
                        'usa_boulders',
                        'usa_boulders_input',
                        'usa_boulders_selector']

checkpoint_df_grade = df['grade'].drop(columns=useless_grade_columns)

In [274]:
#Checkpoint
checkpoint_df_grade.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83 entries, 0 to 82
Data columns (total 1 columns):
id    83 non-null int64
dtypes: int64(1)
memory usage: 1.3 KB


# Merging dataframes

In [275]:
df_merged_train_imp = df_train_imp.merge(checkpoint_df_ascent, left_on='id', right_on='user_id', suffixes=('_user','_ascent'))
len(df_merged_train_imp['user_id'].unique())

14252

In [276]:
# Keep only one user_id column
df_merged_train_imp = df_merged_train_imp.drop(columns=['user_id', 'id_ascent']).rename(columns={'id_user':'user_id'})

# Add an experience column for consistency check
df_merged_train_imp['experience'] = df_merged_train_imp['year'] - df_merged_train_imp['started']
# Keep only those with experience greater than 0 (otherwise bogus)
df_merged_train_imp = df_merged_train_imp[df_merged_train_imp['experience'] >= 0]
# Now drop experience and age_when_started_climbing
df_merged_train_imp = df_merged_train_imp.drop(columns=['age_when_started_climbing', 'experience'])
### NOTE: Deciding to leave the original features (started, birth, year) instead
###       since they may account for the difference in training methods depending
###       on the generation of the user.

In [277]:
# df_merged_imp.to_csv('df_imputed.csv')
print('There are {} unique users in merged_train_imp.'.format(len(df_merged_train_imp['user_id'].unique())))

There are 14241 unique users in merged_train_imp.


In [278]:
df_merged_train_orig = df_train_orig.merge(checkpoint_df_ascent, left_on='id', right_on='user_id', suffixes=('_user','_ascent'))
df_merged_test = df_test.merge(checkpoint_df_ascent, left_on='id', right_on='user_id', suffixes=('_user','_ascent'))

In [279]:
# Keep only one user_id column
df_merged_train_orig = df_merged_train_orig.drop(columns=['user_id', 'id_ascent']).rename(columns={'id_user':'user_id'})
df_merged_test = df_merged_test.drop(columns=['user_id', 'id_ascent']).rename(columns={'id_user':'user_id'})

# Add an experience column for consistency check
df_merged_train_orig['experience'] = df_merged_train_orig['year'] - df_merged_train_orig['started']
df_merged_test['experience'] = df_merged_test['year'] - df_merged_test['started']

# Keep only those with experience greater than 0 (otherwise bogus)
df_merged_train_orig = df_merged_train_orig[df_merged_train_orig['experience'] >= 0]
df_merged_test = df_merged_test[df_merged_test['experience'] >= 0]

# Now drop experience and age_when_started_climbing
df_merged_train_orig = df_merged_train_orig.drop(columns=['age_when_started_climbing', 'experience'])
df_merged_test = df_merged_test.drop(columns=['age_when_started_climbing', 'experience'])

### NOTE: Deciding to leave the original features (started, birth, year) instead
###       since they may account for the difference in training methods depending
###       on the generation of the user.
print('There are {} unique users in merged_train_orig.'.format(len(df_merged_train_orig['user_id'].unique())))
print('There are {} unique users in merged_test.'.format(len(df_merged_test['user_id'].unique())))

There are 11233 unique users in merged_train_orig.
There are 2813 unique users in merged_test.


In [280]:
print("merged_train_orig: {} data points.".format(len(df_merged_train_orig)))
print("merged_train_imp: {} data points.".format(len(df_merged_train_imp)))
print("merged_test: {} data points.".format(len(df_merged_test)))

merged_train_orig: 1758212 data points.
merged_train_imp: 2171996 data points.
merged_test: 432737 data points.


In [281]:
df_merged_train_orig.to_csv('ds_train_orig.csv')
df_merged_train_imp.to_csv('ds_train_imp.csv')
df_merged_test.to_csv('ds_test.csv')